In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install transformers
!pip install peft
!pip install torch
!pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.8 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import torch
import base64
import os
import warnings
import re

# Suppress warnings and set environment variables
warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Check for GPU availability
if not torch.cuda.is_available():
    st.warning("No GPU detected. The model will run on CPU, which may be slower.")

# Load model and adapter
adapter_config_path = "mariam-essam/PytorchModelv2"
adapter_model_path = "mariam-essam/PytorchModelv2"
base_model_name = "unsloth/gemma-7b-bnb-4bit"

base_model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto")
peft_config = PeftConfig.from_pretrained(adapter_config_path)
model = PeftModel.from_pretrained(base_model, adapter_model_path, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

def load_image(image_path):
    try:
        with open(image_path, "rb") as f:
            data = f.read()
            return base64.b64encode(data).decode()
    except FileNotFoundError:
        st.error(f"Image not found at path: {image_path}")
        return ""

background_image = load_image("BG.png")

st.set_page_config(page_title="Medical Chatbot", layout="wide", initial_sidebar_state="expanded")

# Custom CSS for background image and text color
page_bg_css = f"""
<style>
.stApp {{
    background: url(data:image/png;base64,{background_image});
    background-size: cover;
    background-position: center;
    background-attachment: fixed;
    opacity: 0.9;
    color: white;  /* Text color */
    font-family: 'Arial', sans-serif; /* Clear font family */
    font-size: 16px; /* Increase font size for better readability */
    text-shadow: 0px 0px 2px black; /* Subtle shadow for better visibility */
}}
.background {{
    background-color: rgba(0, 0, 0, 0.5);
    padding: 20px;
    border-radius: 10px;
}}
</style>
"""
st.markdown(page_bg_css, unsafe_allow_html=True)

st.title("Medical Assistant Chatbot")
st.markdown("This chatbot is powered by a medical model.")

def clean_response(text):
    """ Clean the model's response to remove HTML tags and repeated sections """
    # Remove any HTML tags if present
    clean_text = re.sub(r'<.*?>', '', text)
    # Remove repeated sentences (basic filtering by splitting sentences)
    sentences = clean_text.split('. ')
    unique_sentences = []
    for sentence in sentences:
        if sentence not in unique_sentences:
            unique_sentences.append(sentence)
    return '. '.join(unique_sentences)

def main():
    if "user_name" not in st.session_state:
        st.session_state.user_name = ""

    if not st.session_state.user_name:
        st.session_state.user_name = st.text_input("Please enter your name to start:")

    if st.session_state.user_name:
        st.markdown(f"**Welcome, {st.session_state.user_name}! Feel free to ask your medical questions.**")

        if "printed_history" not in st.session_state:
            st.session_state.printed_history = []

        for message_type, content in st.session_state.printed_history:
            if message_type == "ai":
                avatar = "🤖"
                name_display = "Health PAL"
            else:
                avatar = "🧑"
                name_display = f"**{st.session_state.user_name}:** "

            st.write(f"{name_display} {content}")

        if len(st.session_state.printed_history) == 0:
            st.write(f"🤖 Hello {st.session_state['user_name']}, I am your Health PAL. How can I assist you today?")

        user_input = st.text_input(f"{st.session_state.user_name}, enter your question:")

        if user_input:
            st.session_state.printed_history.append(("human", user_input))
            st.write(f"🧑 **{st.session_state.user_name}:** {user_input}")

            with st.spinner('Generating Response ...'):
                inputs = tokenizer(user_input, return_tensors="pt").to("cuda")

                # Generate response with some controls to reduce repetition
                output = model.generate(
                    inputs["input_ids"],
                    max_length=100,
                    temperature=0.5,
                    top_p=0.8,
                    repetition_penalty=1.2  # Adjust this penalty to reduce repetition
                )

                response_text = tokenizer.decode(output[0], skip_special_tokens=True)
                # Clean the response before displaying it
                response_text = clean_response(response_text)

                st.session_state.printed_history.append(("ai", response_text))
                st.write(f"🤖 Health PAL: {response_text}")

with st.sidebar:
    st.image("Depi.png", width=100)
    st.title("ChatBot")
    if st.button("➕ New Chat"):
        for key in st.session_state.keys():
            del st.session_state[key]

if __name__ == "__main__":
    main()


Writing app.py


In [ ]:
!ngrok authtoken 2nfKRBwUgeqOPxnSgYcnm2lrAL3_6CvZZq7ssRwTVfc4MTXF6

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Kill any existing ngrok processes
!pkill ngrok

# Start a tunnel to the Streamlit app
public_url = ngrok.connect(8501, "http")
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:8501\"".format(public_url))


 * ngrok tunnel "NgrokTunnel: "https://7ba9-34-142-161-9.ngrok-free.app" -> "http://localhost:8501"" -> "http://127.0.0.1:8501"


In [ ]:
!streamlit run app.py &>/dev/null&
